# আমাদের প্রয়োজনীয় সব ডিপেন্ডেন্সি গুলা ইনস্টল করে নিচ্ছি; ইনস্টল করার পরে সেশন রিস্টার্ট করার জন্য সাজেস্ট করতে পারে, জাস্ট সিম্পলি ইগনোর।


In [ ]:
!pip install pdf2image
!sudo apt-get install poppler-utils
!pip install --upgrade google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2


# আমরা আমাদের ড্রাইভারের সাথে কানেক্ট করে সার্ভিসটা ইনেবল করলাম। এজন্য একটা গুগল অথেনটিকেশন পপ-আপ আসতে পারে, প্লিজ সব গুলা পারমিশন দেন।

In [8]:
from google.colab import auth
import os
import io
from PIL import Image
from pdf2image import convert_from_path
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
auth.authenticate_user()


# এই অংশটুকু কি ভাবে কাজ করছে তার জন্য প্লিজ "README" ফাইল টা read করেন প্লিজ

In [9]:
class DriveOCR:
    def __init__(self):
        self.service = build('drive', 'v3')

    def process_image(self, filepath):
        mime_type = 'application/vnd.google-apps.document'
        file_metadata = {'name': filepath, 'mimeType': mime_type}
        file = self.service.files().create(
            body=file_metadata,
            media_body=MediaFileUpload(filepath, mimetype='image/jpeg'),
            fields='id'
        ).execute()

        file_id = file['id']
        request = self.service.files().export_media(fileId=file_id, mimeType='text/plain')

        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"{filepath}: Download {int(status.progress() * 100)}%")

        self.service.files().delete(fileId=file_id).execute()
        return fh.getvalue().decode()


# step-1: PDF ফাইলে আপলোড করেন এখানে

In [4]:
from google.colab import files
uploaded = files.upload()  # Upload your PDF
pdf_filename = list(uploaded.keys())[0]


Saving Bangla Typing with Avro Phonetic.pdf to Bangla Typing with Avro Phonetic.pdf


# step-2 এখানে দুইটা কাজ হচ্ছে


1.  ফাইল ক্রিয়েট হচ্ছে ইমেজ গুলা সেভ করার জন্য
2.  pdf ফাইল এর প্রতিটি পেজ কে ইমেজ এ কনভার্ট করে ফাইলে সেভ রাখছে



In [5]:
# Create folder to store images
os.makedirs("pdf_pages", exist_ok=True)

# Convert PDF to image list (PIL Images)
images = convert_from_path(pdf_filename)

image_paths = []

for i, img in enumerate(images):
    path = f"pdf_pages/page_{i+1}.jpg"
    img.save(path, "JPEG")
    image_paths.append(path)

print(f"Converted {len(image_paths)} pages to images.")


Converted 17 pages to images.


# step-3: সব গুলো ইমেজ কে এক এক করে ড্রাইভে আপলোড দিচ্ছে আর তা থেকে টেক্সট এক্সট্রাক করে "combined_text" variable এ সেভ রাখছে

In [6]:
ocr = DriveOCR()
combined_text = ""

for path in image_paths:
    print(f"OCR: {path}")
    text = ocr.process_image(path)
    combined_text += f"\n--- Text from {os.path.basename(path)} ---\n{text.strip()}\n"


OCR: pdf_pages/page_1.jpg
pdf_pages/page_1.jpg: Download 100%
OCR: pdf_pages/page_2.jpg
pdf_pages/page_2.jpg: Download 100%
OCR: pdf_pages/page_3.jpg
pdf_pages/page_3.jpg: Download 100%
OCR: pdf_pages/page_4.jpg
pdf_pages/page_4.jpg: Download 100%
OCR: pdf_pages/page_5.jpg
pdf_pages/page_5.jpg: Download 100%
OCR: pdf_pages/page_6.jpg
pdf_pages/page_6.jpg: Download 100%
OCR: pdf_pages/page_7.jpg
pdf_pages/page_7.jpg: Download 100%
OCR: pdf_pages/page_8.jpg
pdf_pages/page_8.jpg: Download 100%
OCR: pdf_pages/page_9.jpg
pdf_pages/page_9.jpg: Download 100%
OCR: pdf_pages/page_10.jpg
pdf_pages/page_10.jpg: Download 100%
OCR: pdf_pages/page_11.jpg
pdf_pages/page_11.jpg: Download 100%
OCR: pdf_pages/page_12.jpg
pdf_pages/page_12.jpg: Download 100%
OCR: pdf_pages/page_13.jpg
pdf_pages/page_13.jpg: Download 100%
OCR: pdf_pages/page_14.jpg
pdf_pages/page_14.jpg: Download 100%
OCR: pdf_pages/page_15.jpg
pdf_pages/page_15.jpg: Download 100%
OCR: pdf_pages/page_16.jpg
pdf_pages/page_16.jpg: Download

# step-4: combined_text, variable এর সব টেক্সট এখন pdf_ocr_output.txt ফাইল এ write করা হচ্ছে

In [7]:
output_file = "pdf_ocr_output.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(combined_text)

files.download(output_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# প্রথমবারের জন্য সবগুলো কোড ব্লক একবার রান করতে হবে। কিন্তু এরপর থেকে নতুন ইমেজের জন্য শুধু Step-1, Step-2, Step-3 and step-4 রিপিট করলেই হবে। 2nd বার থেকে কিন্তু আগের জেনেরেট করা ইমেজ ফাইল "pdf_pages" **DELETE** করে দিতে হবে